# Summarization Using LLM Wrapper

In [1]:
import sys
current_dir = "/Users/josephtolsma/Documents/dev/yelp_rag"
sys.path.insert(0,current_dir)

In [2]:
import pandas as pd
import os
import requests
import src.config as config

In [3]:
pd.set_option("display.max_colwidth",40)

In [4]:
relevant_chunk_df = pd.read_parquet(os.path.join(config.DATA_DIR_PROC,"topic_relevant_review_chunks.parquet"))

In [5]:
relevant_chunk_df.head(3)

,topic,scores,indicies,chunk_id,business_id,restaurant_name,stars,review_id,chunk
0,food,0.512154,69,nE8A-emvMc-8sJ_0_BWYsg_0,IS0B5nLJPluVT8NwGgospA,Cheddar's Scratch Kitchen,5.0,nE8A-emvMc-8sJ_0_BWYsg,This place is ahhmazzing! Every entr...
1,food,0.506828,95,7FzqzVUwsBW_JEjW94fpYQ_0,IS0B5nLJPluVT8NwGgospA,Cheddar's Scratch Kitchen,4.0,7FzqzVUwsBW_JEjW94fpYQ,"My parents were in town visiting, so..."
2,food,0.498265,78,4SDns1Llu0RKzpKZ7w4iKg_0,IS0B5nLJPluVT8NwGgospA,Cheddar's Scratch Kitchen,3.0,4SDns1Llu0RKzpKZ7w4iKg,"This is your usual standard fare, no..."


In [6]:
pd.set_option("display.max_colwidth",None)

**Build Content for Summarization for One Restaurant**

In [7]:
rid = relevant_chunk_df.sample(random_state=42)["business_id"].values[0]

In [8]:
chunk_test_df = relevant_chunk_df[relevant_chunk_df["business_id"] == rid]

In [9]:
topic_context_dict = {}

for topic in config.TOPICS.keys():
    chunk_topic_df = chunk_test_df[chunk_test_df["topic"]==topic]
    context = f"""
    You are analyzing and synthesizing customer reviews for a restaurant called {chunk_test_df["restaurant_name"].values[0]}.

    The following are excerpts from different customer reviews. 
    Each review comes from a different reviewer.

    """
    context+=f"""
    Topic: {topic.upper()}
    """
    i = 0
    for row in chunk_topic_df.iterrows():

        verbatim = f"""
        [{i}] Review ({row[1].stars} stars):
        {row[1].chunk}
        """
        context+=verbatim
        i+=1
    topic_context_dict.update({topic:context})

In [10]:
def call_ollama(
        prompt,
        model,
        temperature=0.2
    ):
    
    url = "http://localhost:11434/api/generate"
    payload = {
        "model":model,
        "prompt":prompt,
        "stream":False,
        "options":{"temperature":temperature}
    }

    resp = requests.post(url,json=payload,timeout=120)
    resp.raise_for_status()
    
    data = resp.json()
    return data["response"].strip()

In [11]:
print(call_ollama(prompt = "say hello.",model = config.OLLAMA_MODEL,temperature=0.2))

Hello! How are you today?


In [12]:
summary_dict = {}
for topic in config.TOPICS.keys():
    chunk_topic_df = chunk_test_df[chunk_test_df["topic"]==topic]
    restaurant_name = chunk_test_df["restaurant_name"].values[0]
    context = f"""
    You are analyzing and synthesizing customer reviews for a restaurant called {restaurant_name}, where the main topic is:
    
    Topic: {topic.upper()}

    Using only the reviews included here, provide a concise (2-3 sentence) summary of {restaurant_name}'s performance in the area of {topic}.

    Guidelines:
    - Capture common themes and disagreements
    - Mention both positives and negatives if present
    - Be factual and grounded in the evidence
    - Do not invent details not supported by the excerpts
    - Do not announce the summary ("here is a concise summary..."); just start with summary content

    """
    i = 0
    for row in chunk_topic_df.iterrows():

        verbatim = f"""
        [{i}] Review ({row[1].stars} stars):
        {row[1].chunk}
        """
        context+=verbatim
        i+=1
    
    summary = call_ollama(prompt = context,
                          model = config.OLLAMA_MODEL,
                          temperature=config.TEMPERATURE
                        )
    summary_dict.update({topic:summary})

In [13]:
pd.DataFrame(summary_dict,index = [0])

,food,service,ambiance
0,"Ants Pants Café's food offerings receive mixed reviews, but overall, customers praise the quality of certain dishes such as sweet potato fries, egg-based entrees (e.g., eggs benedict, omelets), and brunch specials. However, some reviewers express disappointment with other menu items, citing underwhelming flavors or poorly executed dishes like dill-scrambled eggs. The café's ability to execute consistent flavors is questioned, but the quality of ingredients and presentation are generally commended.","Ants Pants Café's service has been inconsistent, with some reviewers experiencing friendly and attentive waitstaff, while others have reported rude, indifferent, or incompetent service. Some common complaints include long wait times, slow service, and a lack of attention to customers' needs. However, a few reviewers have had positive experiences with friendly staff, suggesting that the issue may be more related to staffing levels or training rather than a fundamental problem with the café's culture.","Ants Pants Café's ambiance is generally praised as cozy and welcoming, with reviewers noting the excellent music setting, bright lighting, and ""cute"" or ""low key"" atmosphere. However, some reviewers criticize the high noise levels, particularly when large groups of loud patrons are present, which can detract from the dining experience. The small size of the café is also sometimes seen as a drawback, leading to long wait times and overcrowding during peak hours."


**Build Content for Summarization for All Restaurants**

In [14]:
restaurant_ids = relevant_chunk_df["business_id"].unique().tolist()

In [20]:
summary_dict = {}
for id in restaurant_ids:
    restaurant_df = relevant_chunk_df[relevant_chunk_df["business_id"] == id]
    restaurant_name = restaurant_df.iloc[0]["restaurant_name"]
    restaurant_summary_dict = {}
    for topic in config.TOPICS.keys():
        topic_df = restaurant_df[restaurant_df["topic"]==topic]
        context = f"""
        You are analyzing and synthesizing customer reviews for a restaurant called {restaurant_name}, where the main topic is:
        
        Topic: {topic.upper()}

        Using only the reviews included here, provide a concise (2-3 sentence) summary of {restaurant_name}'s performance in the area of {topic}.

        Guidelines:
        - Capture common themes, impressions, and disagreements
        - Be factual and grounded in the evidence; do not invent details not supported by the excerpts
        - Begin by characterizing the group of reviews as "positive", "negative", or "mixed" 

        Output to avoid:
        - Content unrelated to the topic (for example: "noise levels" are ambiance, not food)
        - Review indicies (for example, "review 3"); those are not user-facing
        - Announcing the summary ("here is a concise summary..."); just start with summary content
        - Use of generic filler adjectives (e.g., cozy, delicious, amazing) unless the excerpts explicitly support them

        Important: if the excerpts don’t describe a topic clearly, say ‘<topic> details are limited in the provided reviews.’”

        """
        i = 0
        for row in chunk_topic_df.iterrows():

            verbatim = f"""
            [{i}] Review ({row[1].stars} stars):
            {row[1].chunk}
            """
            context+=verbatim
            i+=1
        
        summary = call_ollama(prompt = context,
                            model = config.OLLAMA_MODEL,
                            temperature=config.TEMPERATURE
                            ).replace("\n"," ")
        restaurant_summary_dict.update({topic:summary})
    summary_dict.update({restaurant_name:restaurant_summary_dict})

In [21]:
pd.DataFrame(summary_dict).T

,food,service,ambiance
Cheddar's Scratch Kitchen,"The reviews are mixed, with some reviewers praising the high quality food and others finding it unexceptional. Reviewers rave about specific dishes such as the pumpkin pancakes, sweet potato fries, eggs Benedict, brie stuffed French toast, and spinach goat cheese omelette, suggesting that Cheddar's Scratch Kitchen has a strong menu. However, one reviewer found their meal to be merely ""good"" and not memorable. Some reviewers also mention issues with noise levels, with two reviewers complaining about the high noise level due to hard surfaces and loud music.","The reviews are mixed regarding the service at Cheddar's Scratch Kitchen. Some reviewers, such as those with 5-star ratings, praise the outgoing and superb service, while others with lower ratings mention that the staff is friendly but not exceptional. A few reviewers specifically mention that the noise level can be a distraction from the service, but it does not seem to negatively impact their overall experience. Overall, the service seems to be good but not outstanding.","The reviews are mixed, with some customers praising the cozy atmosphere and others finding it too loud or ""posey"". Some reviewers mention that the small space can be charming, but also leads to long wait times and a lack of seating. However, the majority of reviewers seem to appreciate the restaurant's ambiance, with many using words like ""cozy"", ""cute"", and ""charming"" to describe it."
French Toast,"The reviews are mixed, but overall French Toast's food performance is generally positive. Many reviewers praise the high-quality ingredients and creative dishes, such as the brie stuffed french toast with apricot glaze and the sweet potato fries. However, some reviewers mention that the food can be inconsistent, with one reviewer stating that their Crème Brulee Battered French Toast was ""just ok."" Additionally, there is a common theme of noise levels being too high in the restaurant, which may detract from the dining experience.","The service at French Toast is generally positive, with many reviewers praising the friendly and attentive staff. Reviewers mention that servers are ""extremely nice"", ""outgoing and superb"", and provide regular check-ins during busy periods. Some reviewers note that the noise level can be a issue due to the small space and hard surfaces, but this does not seem to impact the service quality.","The reviews are mixed, with some praising the cozy and charming atmosphere, while others criticize the noise level and overcrowding. Some reviewers describe the ambiance as ""cozy"", ""charming"", and ""cute"", with one reviewer specifically mentioning that it feels like a Manhattan LES cafe. Others appreciate the eclectic decor, such as Ikea furniture. However, several reviews mention that the noise level is a problem, with some describing it as ""unbearable"" and others saying it's due to hard surfaces reflecting sound. One reviewer even mentions that the atmosphere can be ruined by annoying hipsters. Overall, while French Toast has its fans who appreciate its unique atmosphere, there are also many who find it overwhelming or unpleasant due to noise and overcrowding issues."
Spasso Italian Grill,"The reviews are mixed, but overall, Spasso Italian Grill receives praise for its food. Many reviewers rave about the quality and taste of their dishes, with some specifically mentioning that they have the best pancakes or sweet potato fries in the neighborhood. However, a few reviewers mention that their experience was marred by loud noise levels, which can make it difficult to hear tablemates.","The service at Spasso Italian Grill is generally positive, with many reviewers praising the friendly and attentive staff. Some reviewers mention that the service can be slow during peak hours, but overall, the staff's hospitality and ability to accommodate customers' needs are notable. However, a few reviewers express frustration with noise lev